# Model - YOLOv8
# type - one vs all
# Dataset - VisDrone 
# Test - extended (classwise)

# 1. Install Dependencies


In [1]:
!pip install ultralytics
!pip install torch torchvision

import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 599.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 6.2 MB/s eta 0:00:00


# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image


# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if len(annotation) != 8:
#                 print(f"Error parsing annotation in {filename}: {annotation}")
#                 print(f"Ignoring malformed annotation: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            center_x = x_min + (width / 2.0)
            center_y = y_min + (height / 2.0)
            center_x /= img_width
            center_y /= img_height
            width /= img_width
            height /= img_height
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


## for train, val from visdrone

In [3]:
def create_files_train(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images'
    os.makedirs( '/kaggle/working/yolo_dataset/train', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/train')

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'train/labels')
    output_images_path = os.path.join(output_dataset_path, 'train/images')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [4]:
def create_files_val(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images'
    os.makedirs( '/kaggle/working/yolo_dataset/val', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/val')

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'val/labels')
    output_images_path = os.path.join(output_dataset_path, 'val/images')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

## for test from expanded

In [5]:
def create_files_test(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/extended-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/extended-dataset/Expanded_dataset/images'
    
    os.makedirs( '/kaggle/working/yolo_dataset/test', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/test')

    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'test/labels')
    output_images_path = os.path.join(output_dataset_path, 'test/images')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [6]:
def classes_label(class_name):
    create_files_train(class_name)
    create_files_val(class_name)
    create_files_test(class_name)


In [7]:
# classes_label(0)

# 3. Import yolov8

In [8]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO('/kaggle/working/yolov8n.pt')  # You can also use yolov8m.pt, yolov8l.pt, etc.

100%|██████████| 6.23M/6.23M [00:00<00:00, 70.9MB/s]


# 4. Train Class wise



# Class 0 - pedestrian

In [9]:
classes_label(0)

In [10]:
data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['pedestrian']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)

    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='pedestrian_1')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolov8n.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]
2024-06-27 15:52:46,806	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-27 15:52:49,248	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 1106 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:05<00:00, 1219.07it/s]

train: WARNING ⚠️ /kaggle/working/yolo_dataset/train/images/0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /kaggle/working/yolo_dataset/train/images/9999987_00000_d_0000049.jpg: 1 duplicate labels removed


...

      15/15      5.71G      2.164      1.416     0.9455         24        640: 100%|██████████| 405/405 [01:26<00:00,  4.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  4.75it/s]

                   all        548       8844      0.496      0.339      0.352       0.15



15 epochs completed in 0.394 hours.
Optimizer stripped from runs/detect/pedestrian_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_1/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:08<00:00,  2.19it/s]


                   all        548       8844      0.497      0.338      0.352       0.15
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/pedestrian_1


In [11]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:08<00:00,  3.99it/s]


                   all        548       8844        0.5      0.339      0.354      0.151
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/pedestrian_12


In [12]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_1/weights/best.pt')

# Define the new data.yaml file path
new_data_path = '/kaggle/working/yolo_dataset/data.yaml'

# Evaluate the model on the new dataset
test_results = model.val(data=new_data_path, split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 372 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:01<00:00, 1158.11it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:13<00:00,  6.04it/s]


                   all       1301      13460      0.302     0.0897     0.0849     0.0401
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val


# Class 1 - People

In [13]:
classes_label(1)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['people']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)

In [14]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='people_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=people_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 2524 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1298.55it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 66 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1088.44it/s]


...

      15/15      2.78G      2.516      2.002     0.9625          2        640: 100%|██████████| 405/405 [01:25<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.30it/s]

                   all        548       5125       0.45      0.314      0.307       0.11



15 epochs completed in 0.393 hours.
Optimizer stripped from runs/detect/people_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/people_1/weights/best.pt, 6.2MB

Validating runs/detect/people_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:09<00:00,  1.97it/s]


                   all        548       5125       0.45      0.314      0.307       0.11
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/people_1


In [15]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 66 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.73it/s]


                   all        548       5125      0.452      0.317       0.31      0.111
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/people_12


In [16]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/people_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 708 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1367.62it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:08<00:00,  9.42it/s]


                   all       1301       4438      0.049     0.0665     0.0205    0.00582
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val2


# class 2 - bicycle

In [17]:
classes_label(2)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['bicycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [18]:

import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='bicycle_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/people_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=bicycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 3717 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1334.98it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 184 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1317.39it/s]

...

                   all        548       1287      0.188      0.143     0.0735     0.0259

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15      2.58G      2.658      2.717       1.09          0        640: 100%|██████████| 405/405 [01:22<00:00,  4.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.63it/s]


                   all        548       1287      0.182      0.159     0.0824     0.0278

15 epochs completed in 0.391 hours.
Optimizer stripped from runs/detect/bicycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/bicycle_1/weights/best.pt, 6.2MB

Validating runs/detect/bicycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  4.73it/s]


                   all        548       1287      0.185      0.159     0.0826     0.0279
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/bicycle_1


In [19]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 184 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:04<00:00,  7.32it/s]


                   all        548       1287      0.187      0.159     0.0824     0.0281
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/bicycle_12


In [20]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/bicycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 1032 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1604.84it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:07<00:00, 10.82it/s]


                   all       1301        828    0.00163      0.087   0.000932   0.000316
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val3


# class 3 - car

In [21]:
classes_label(3)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['car']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [22]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='car_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/bicycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=car_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 339 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:05<00:00, 1135.70it/s]

train: WARNING ⚠️ /kaggle/working/yolo_dataset/train/images/0000140_00118_d_0000002.jpg: 1 duplicate labels removed


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15      4.43G      1.267     0.8038     0.9637         83        640: 100%|██████████| 405/405 [01:24<00:00,  4.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.33it/s]


                   all        548      14064      0.774      0.609       0.69      0.444

15 epochs completed in 0.396 hours.
Optimizer stripped from runs/detect/car_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/car_1/weights/best.pt, 6.2MB

Validating runs/detect/car_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:11<00:00,  1.54it/s]


                   all        548      14064      0.773       0.61       0.69      0.444
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/car_1


In [23]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 33 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   9%|▊         | 3/35 [00:03<00:36,  1.14s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:13<00:00,  2.54it/s]


                   all        548      14064      0.774      0.607      0.688      0.443
Speed: 0.2ms preprocess, 4.4ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/car_12


In [24]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/car_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 86 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:01<00:00, 1040.45it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:15<00:00,  5.17it/s]


                   all       1301      16619      0.592       0.47      0.471      0.162
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val4


# Class 4 - van

In [25]:
classes_label(4)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['van']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [26]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='van_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/car_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=van_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 1524 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1406.92it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 127 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1510.43it/s]

...

                   all        548       1975      0.456      0.317      0.329      0.215

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15       2.6G      1.295      1.444     0.9806         49        640: 100%|██████████| 405/405 [01:22<00:00,  4.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.18it/s]


                   all        548       1975      0.468      0.324      0.332      0.218

15 epochs completed in 0.383 hours.
Optimizer stripped from runs/detect/van_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/van_1/weights/best.pt, 6.2MB

Validating runs/detect/van_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  3.76it/s]


                   all        548       1975      0.468      0.323      0.331      0.218
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/van_1


In [27]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 127 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  6.72it/s]


                   all        548       1975      0.468      0.323      0.331      0.218
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/van_12


In [28]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/van_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 453 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1394.21it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:09<00:00,  8.58it/s]


                   all       1301       3415      0.288      0.268      0.198      0.117
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val5


# Class 5 - truck

In [29]:
classes_label(5)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['truck']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [30]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='truck_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/van_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=truck_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 2921 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1594.13it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 282 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1791.12it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15      2.52G      1.448      1.662      1.124          5        640: 100%|██████████| 405/405 [01:23<00:00,  4.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.78it/s]


                   all        548        750      0.431      0.236      0.245      0.157

15 epochs completed in 0.380 hours.
Optimizer stripped from runs/detect/truck_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/truck_1/weights/best.pt, 6.2MB

Validating runs/detect/truck_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.13it/s]


                   all        548        750      0.431      0.236      0.245      0.156
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/truck_1


In [31]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 282 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:04<00:00,  8.58it/s]


                   all        548        750      0.431      0.236      0.245      0.156
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/truck_12


In [32]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/truck_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 813 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1498.78it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:08<00:00,  9.97it/s]


                   all       1301       1402      0.206      0.183       0.11     0.0569
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val6


# Class 6 - tricycle

In [33]:
classes_label(6)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [34]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='tricycle_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/truck_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tricycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 4784 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:03<00:00, 1669.75it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 211 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1339.94it/s]

...

      15/15      2.44G      2.034      2.635      1.099          2        640: 100%|██████████| 405/405 [01:19<00:00,  5.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.03it/s]

                   all        548       1045      0.343      0.227      0.184     0.0904



15 epochs completed in 0.367 hours.
Optimizer stripped from runs/detect/tricycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/tricycle_1/weights/best.pt, 6.2MB

Validating runs/detect/tricycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  4.66it/s]


                   all        548       1045      0.346      0.229      0.184     0.0905
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/tricycle_1


In [35]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 211 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:04<00:00,  8.06it/s]


                   all        548       1045      0.346      0.229      0.184     0.0917
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/tricycle_12


In [36]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 1133 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1755.57it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:07<00:00, 11.38it/s]


                   all       1301        338      0.066     0.0473     0.0203    0.00906
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val7


# Class 7 - awning-tricycle

In [37]:
classes_label(7)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['awning-tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [38]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='awning-tricycle_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/tricycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=awning-tricycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 5321 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:03<00:00, 1739.72it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 328 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1738.26it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15       2.4G      1.792       2.83      1.013         10        640: 100%|██████████| 405/405 [01:20<00:00,  5.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.97it/s]


                   all        548        532      0.262      0.139     0.0917     0.0508

15 epochs completed in 0.367 hours.
Optimizer stripped from runs/detect/awning-tricycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/awning-tricycle_1/weights/best.pt, 6.2MB

Validating runs/detect/awning-tricycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  4.57it/s]


                   all        548        532      0.251      0.128     0.0955      0.054
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/awning-tricycle_1


In [39]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 328 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:04<00:00,  8.01it/s]


                   all        548        532      0.251      0.128     0.0955     0.0536
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/awning-tricycle_12


In [40]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/awning-tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1557.91it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:07<00:00, 10.77it/s]


                   all       1301        390     0.0816     0.0436     0.0167    0.00481
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val8


# Class 8 - bus

In [41]:
classes_label(8)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['bus']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [42]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='bus_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/awning-tricycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=bus_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 4448 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1501.17it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 548 images, 417 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1510.56it/s]

...

      15/15      2.35G      1.347      1.589      1.084          2        640: 100%|██████████| 405/405 [01:26<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.02it/s]

                   all        548        251      0.727      0.372      0.447      0.282



15 epochs completed in 0.435 hours.
Optimizer stripped from runs/detect/bus_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/bus_1/weights/best.pt, 6.2MB

Validating runs/detect/bus_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.36it/s]


                   all        548        251      0.725      0.378      0.447      0.282
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/bus_1


In [43]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 417 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:03<00:00,  9.21it/s]


                   all        548        251      0.724      0.378      0.447      0.282
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/bus_12


In [44]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/bus_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 718 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1387.16it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:09<00:00,  8.95it/s]


                   all       1301       1640      0.498      0.337      0.345      0.227
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val9


# Class 9 - motor

In [45]:
classes_label(9)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['motor']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [46]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='motor_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/bus_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=motor_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 2235 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:04<00:00, 1308.26it/s]

train: WARNING ⚠️ /kaggle/working/yolo_dataset/train/images/9999945_00000_d_0000114.jpg: 1 duplicate labels removed


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      15/15      3.17G      2.246      1.828      1.001          3        640: 100%|██████████| 405/405 [01:22<00:00,  4.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.98it/s]

                   all        548       4886      0.465      0.362      0.351      0.141



15 epochs completed in 0.389 hours.
Optimizer stripped from runs/detect/motor_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/motor_1/weights/best.pt, 6.2MB

Validating runs/detect/motor_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:07<00:00,  2.33it/s]


                   all        548       4886      0.464      0.363      0.351      0.141
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/motor_1


In [47]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 63 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:06<00:00,  5.18it/s]


                   all        548       4886      0.464      0.363      0.351      0.142
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/motor_12


In [48]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/motor_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 724 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1532.16it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:08<00:00,  9.73it/s]


                   all       1301       3944     0.0486     0.0266     0.0165    0.00653
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val10


# Class 10- others

In [49]:
classes_label(10)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['others']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [50]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='others_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/motor_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=others_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning /kaggle/working/yolo_dataset/train/labels... 6471 images, 5680 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:03<00:00, 1735.36it/s]

train: WARNING ⚠️ /kaggle/working/yolo_dataset/train/images/9999998_00219_d_0000175.jpg: 1 duplicate labels removed


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      15/15      2.38G      1.913      3.139      1.059          3        640: 100%|██████████| 405/405 [01:19<00:00,  5.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.60it/s]

                   all        548         32    0.00226      0.281    0.00595    0.00349



15 epochs completed in 0.366 hours.
Optimizer stripped from runs/detect/others_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/others_1/weights/best.pt, 6.2MB

Validating runs/detect/others_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.39it/s]


                   all        548         32    0.00226      0.281    0.00598    0.00351
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/others_1


In [51]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 548 images, 519 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:03<00:00, 10.55it/s]


                   all        548         32    0.00226      0.281    0.00593    0.00347
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/others_12


In [52]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/others_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1301 images, 1196 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1840.41it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:06<00:00, 11.75it/s]


                   all       1301        171      0.688      0.026     0.0345     0.0149
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val11
